In [1]:
# import numpy as np
# import pandas as pd
# from surprise import SVD, Dataset, Reader
# from surprise.model_selection import train_test_split

## Load Data

In [1]:
import yaml
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
import recmetrics
from src.lightgcn import LightGCN
from src.train_lightgcn import *
# from src.utils_v2 import *
# pd.set_option('display.max_colwidth', None)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device # = "cpu"

device(type='cpu')

In [3]:
# Load Config
%cd ..
with open("config.yaml") as config_file:
    config = yaml.safe_load(config_file)

/Users/yingkang/4thBrain/GNN-eCommerce


In [15]:
newdf = pd.read_csv(config['data']['preprocessed'] + "u_i_weight_0.01_0.1_-0.09.csv")
newdf

,Unnamed: 0,user_id,item_id,weight
0,0,465496,5769989,0.01
1,1,465496,5865524,0.01
2,2,465496,5865526,0.01
3,3,1120748,5240,0.01
4,4,1180452,5881337,0.01
...,...,...,...,...
10157403,10157403,622090043,5850628,0.01
10157404,10157404,622090052,5688691,0.01
10157405,10157405,622090052,5931986,0.01
10157406,10157406,622090098,5650609,0.01


## Load saved best LGCN model

In [5]:
csv_path = config['data']['preprocessed'] + "interaction_matrix.csv"
checkpoint_dir = "model-checkpoints"

In [9]:
test_LGCN = TrainLightGCN(csv_path, checkpoint_dir)

n_users :  1589341 , n_items :  54412
train_df Size  :  9649537
val_pos_list_df Size :  22170
test_pos_list_df Size :  22071


In [10]:
tune_config = {
    "latent_dim": 80,
    "n_layers": 3,
    "LR": 0.005,
    "DECAY": 0.0001,  # reg loss
    "BATCH_SIZE": 1024,  # train mini batch size
}

In [11]:
best_model = torch.load(checkpoint_dir + "/2023-02-13_203014/LightGCN_best.pt",
                        map_location=torch.device('cpu'))
best_epoch = best_model['epoch']
best_val_precision = best_model['precision']
best_val_recall = best_model['recall']

test_model = LightGCN(test_LGCN.n_users+test_LGCN.n_items, tune_config["latent_dim"], tune_config["n_layers"])
test_model.load_state_dict(best_model['model_state_dict'])

RuntimeError: Error(s) in loading state_dict for LightGCN:
	size mismatch for embedding.weight: copying a param with shape torch.Size([1592523, 80]) from checkpoint, the shape in current model is torch.Size([1643753, 80]).

In [ ]:
test_p, test_recall = test_LGCN.test(test_model, test_LGCN.test_pos_list_df, test_LGCN.test_interactions_t, K=20)
print(f"Best epoch {best_epoch}")
print(f"Test Precision: {test_p:>0.4f}, Recall: {test_recall:>0.4f}")
print(f"Val Precision: {best_val_precision:>0.4f}, Recall: {best_val_recall:>0.4f}")

## Maybe not -- Long Tail Plot for item popularity and active user

In [ ]:
# fig = plt.figure(figsize=(15, 7))
# recmetrics.long_tail_plot(df=interaction_matrix,
#              item_id_column="item_id",
#              interaction_type="received interests",
#              percentage=0.5,
#              x_labels=False)

In [1]:
# interaction_matrix['item_id'].value_counts()

Some item is extremly popular(massive user shows interest to it)

### MAR@K Plot

In [ ]:
mark_scores = [random_mark, pop_mark, cf_mark]
index = range(1,10+1)
names = ['Random Recommender', 'Popularity Recommender', 'Collaborative Filter']

fig = plt.figure(figsize=(15, 7))
recmetrics.mark_plot(mark_scores, model_names=names, k_range=index)

## Hit Rate

## ROC Plot + Precision Recall Plot

## Average Precision

## Prediction Coverage + Coverage Plot